# Laboratorio 6 Parte 2

### Reducción de dimensión: PCA y LDA

### 2019-I

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co

Modificado por Heisman Arcila. (Versión original en: https://github.com/jdariasl/ML_IntroductoryCourse/tree/master/Labs)

#### Primer Integrante: Danny Hernandez
#### C.C: 1105784633
#### Segundo Integrante: Daniel Martinez
#### C.C:1036645337

## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

## Indicaciones

Este ejercicio tiene como objetivo implementar varias técnicas de extracción de características (PCA y LDA) y usar SVM para resolver un problema de clasificación multietiqueta o multiclase.

Para el problema de clasificación usaremos la siguiente base de datos: https://archive.ics.uci.edu/ml/datasets/Cardiotocography



Analice la base de datos, sus características, su variable de salida y el contexto del problema.

Antes de iniciar a ejecutar las celdas, debe instalar la librería mlxtend que usaremos para los laboratorios de reducción de dimensión.
Para hacerlo solo tiene que usar el siguiente comando: sudo pip install mlxtend. También puede consultar la guía oficial de instalación
    de esta librería: https://rasbt.github.io/mlxtend/installation/

Analice y comprenda la siguiente celda de código donde se importan las librerías a usar y se carga la base de datos.

In [0]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from mlxtend.preprocessing import standardize
from mlxtend.feature_extraction import PrincipalComponentAnalysis as PCA
from mlxtend.feature_extraction import LinearDiscriminantAnalysis as LDA
import warnings
warnings.filterwarnings("ignore")
import time

#cargamos la bd de entrenamiento
db = np.loadtxt('/content/DB_Fetal_Cardiotocograms(1).txt',delimiter='\t')  # Assuming tab-delimiter

X = db[:,0:22]

#Solo para dar formato a algunas variables
for i in range(1,7):
    X[:,i] = X[:,i]*1000

X = X
Y = db[:,22]

#Para darle formato de entero a la variable de salida

Y_l = []
for i in Y:
    Y_l.append(int(i))
Y = np.asarray(Y_l)

print ("Dimensiones de la base de datos de entrenamiento. dim de X: " + str(np.shape(X)) + "\tdim de Y: " + str(np.shape(Y)))


Dimensiones de la base de datos de entrenamiento. dim de X: (2126, 22)	dim de Y: (2126,)


En la siguiente celda de código no tiene que completar nada. Analice, comprenda y ejecute el código y tenga en cuenta los resultados para completar la tabla que se le pide más abajo.

In [0]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

#Para calcular el costo computacional
tiempo_i = time.time()

#Creamos el clasificador SVM. Tenga en cuenta que el problema es multiclase. 
clf = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)

#Implemetamos la metodología de validación

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  

    #Aquí se entran y se valida el modelo sin hacer selección de características
    
    ######
    
    # Entrenamiento el modelo.
    model = clf.fit(X_train,y_train)

    # Validación del modelo
    ypred = model.predict(X_test)
    
    #######

    Errores[j] = classification_error(ypred, y_test)
    j+=1

print("\nError de validación sin aplicar extracción: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")

#print str(ypred)
#print str(y_test) 


Error de validación sin aplicar extracción: 0.07712817787226504 +/- 0.05442325724156325


Tiempo total de ejecución: 0.607682466506958 segundos.


## Ejercicio 1

1.1 Al aplicar PCA es necesario estandarizar los datos? Si, No y por qué? En qué consiste dicha estandarización?

R/:  Si, por que los datos deben estar centrados y esto se hace restando la media. Estandarizar las variables para que tengan media 0 y desviación estándar 1, ya que de lo contrario, las variables con mayor varianza dominarían al resto.
    
1.2 La proyección de los datos que realiza PCA busca optimizar un medida, ¿Cuál? Explique.

R/: Busca la proyeccion de los datos en un espacio donde la variabilidad de los datos es maxima varianza buscando asi encontrar el valor de los pesos para esa maxima.

## Ejercicio 2

En la siguiente celda, complete el código donde le sea indicado. Consulte la documentación oficial de la librería mlxtend para los métodos de extracción de características. https://rasbt.github.io/mlxtend/user_guide/feature_extraction/

In [0]:
#Feature Extraction Function
#Recibe 2 parámetros: 1. el tipo de método de extracción (pca o lda como string), 2. el número componentes (para pca)
#o el número de discriminantes (para lda)

#Para este laboratorio solo se le pedirá trabajar con PCA, LDA es opcional.

def extract_features(tipo, n):
    
    if tipo == 'pca':
    
        ext = PCA(n_components=n)
    
        return ext

    elif tipo == 'lda':
        
        ext = LDA(n_discriminants=n)
        
        return ext
    
    else:
        print ("Ingrese un método válido (pca o lda)\n")

def invento(tipazo, numero):
  #Para calcular el costo computacional
  tiempo_i = time.time()
  global X
  global Y

  #Estandarizamos los datos
  X = standardize(X)

  #Implemetamos la metodología de validación cross validation con 10 folds

  Errores = np.ones(10)
  j = 0
  kf = KFold(n_splits=10)

  for train_index, test_index in kf.split(X):

      #Aquí se aplica la extracción de características por PCA
      #Complete el código

      ex = extract_features(tipazo, numero)#Complete el código llamando el método extract_features. Tenga en cuenta lo que le pide el ejercicio 3.1

      #Fit de PCA
      ex = ex.fit(X)#Complete el código con el fit correspondiente

      #Transforme las variables y genere el nuevo espacio de características de menor dimensión
      X_ex = ex.transform(X)#complete el código aquí para hacer la transformación


      #Aquí se aplica la extracción de características por LDA

      #OPCIONAL
      '''
      ex = #Complete el código llamando el método extract_features.Tenga en cuenta lo que le pide el ejercicio 3.1

      #Fit de LDA
      ex = #Complete el código con el fit correspondiente

      #Transforme las variables y genere el nuevo espacio de características de menor dimensión
      X_ex = #complete el código aquí para hacer la transformación
      '''

      #Se aplica CV-10

      X_train, X_test = X_ex[train_index], X_ex[test_index]
      y_train, y_test = Y[train_index], Y[test_index]  

      #Aquí se entrena y se valida el modelo luego de aplicar extracción de características con PCA o LDA

      ######

      # Entrenamiento el modelo.
      model = clf.fit(X_train,y_train)

      # Validación del modelo
      ypred = model.predict(X_test)

      #######

      Errores[j] = classification_error(ypred, y_test)
      j+=1


  print("\nError de validación aplicando extracción: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

  print("\nEficiencia en validación aplicando extracción: " + str((1-np.mean(Errores))*100) + "%" )
  tiempizimo=time.time()-tiempo_i
  print ("\n\nTiempo total de ejecución: " + str(tiempizimo) + " segundos.")

  #print str(ypred)
  #print str(y_test)
  return np.mean(Errores), np.std(Errores), tiempizimo

## Ejercicio 3

3.1 En la celda de código anterior, varíe los parámetros correspondientes al número de componentes principales a tener en cuenta (use 2, 10, 19 y 21 componentes principales) para PCA y complete la siguiente tabla de resultados:

In [0]:
import pandas as pd
#import qgrid
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['SVM sin extracción','SVM + PCA','SVM + PCA','SVM + PCA','SVM + PCA']),
    '# de características seleccionadas' : pd.Series(['N/A',2,10,19,21]),
   })
df_types["Error de validación"] = ""
df_types["IC(std)"] = ""
df_types["Tiempo de ejecución"] = ""

df_types.set_index(['Tecnica','# de características seleccionadas'], inplace=True)
for i in df_types.index:
    print(i[1])
    if i[1]=='N/A':
      
      ED=invento('pca', None)
      df_types["Error de validación"][i] = float(ED[0])
      df_types["IC(std)"][i] = float(ED[1])
      df_types["Tiempo de ejecución"][i] = float(ED[2])
    else:
      ED=invento('pca', i[1])
      df_types["Error de validación"][i] = float(ED[0])
      df_types["IC(std)"][i] = float(ED[1])
      df_types["Tiempo de ejecución"][i] = float(ED[2])

#df_types.sort_index(inplace=True)
#qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
#qgrid_widget

N/A

Error de validación aplicando extracción: 0.07336345114713438 +/- 0.043533331744924374

Eficiencia en validación aplicando extracción: 92.66365488528656%


Tiempo total de ejecución: 0.712254524230957 segundos.
2

Error de validación aplicando extracción: 0.22142793870139074 +/- 0.17001475908401714

Eficiencia en validación aplicando extracción: 77.85720612986093%


Tiempo total de ejecución: 0.863616943359375 segundos.
10

Error de validación aplicando extracción: 0.09169324120825582 +/- 0.061564467997647386

Eficiencia en validación aplicando extracción: 90.83067587917442%


Tiempo total de ejecución: 0.6724224090576172 segundos.
19

Error de validación aplicando extracción: 0.07193949862698203 +/- 0.04841386304281984

Eficiencia en validación aplicando extracción: 92.8060501373018%


Tiempo total de ejecución: 0.7097976207733154 segundos.
21

Error de validación aplicando extracción: 0.07430463282841704 +/- 0.04391645244688941

Eficiencia en validación aplicando extracción: 92.

#No funciona QGRID

In [0]:
df_types

Error de validación  ... Tiempo de ejecución
Tecnica            # de características seleccionadas                      ...                    
SVM sin extracción N/A                                          0.0733635  ...            0.712255
SVM + PCA          2                                             0.221428  ...            0.863617
                   10                                           0.0916932  ...            0.672422
                   19                                           0.0719395  ...            0.709798
                   21                                           0.0743046  ...            0.703228

[5 rows x 3 columns]

3.2 Analizando los resultados del punto anterior que puede decir de la viabilidad de aplicar PCA para hacer reducción de dimensión en este problema?

R/: Es viable  ya que con 19 caracteristicas nos da un error en la validacion del 7.19% +- 4.84%, se le estarian quitando 3 de las que tiene originalmente.


3.3 Explique en sus palabras la principal ventaja que tiene LDA sobre PCA para resolver problemas de clasificación.

R/: LDA tiene la ventaja de realizar una proyección respecto a las medias de cada máximo de su clase respecto a la otra, es decir tiene mejor separacion de proyección.


3.3 Explique en sus palabras las diferencias que existen entre los métodos de selección de características y los métodos de extracción de características vistos en el curso.

R/: La selección de caracterizticas busca las caracteristicas de mayor importancia del problema que se esta abordando sin que su presición o predicción disminuya. La extracción de caracteristicas reduce el problema, es decir acorta su dimensión y con estas busca las proyecciones sobre un espacio menor, es decir transforma los datos o crea datos.